In [35]:
import pandas as pd
import folium
import xyzservices.providers as xyz
MAPBOX_API_KEY = open('../../mapbox_api_key', 'r').read()
from PIL import Image
import requests
from io import BytesIO
import urllib.request
import base64
import os, os.path

In [36]:
path = "../../cropped_images"
for f in os.listdir(path):
    print(os.path.join(path, f))
imgs = [Image.open(os.path.join(path, f)) for f in os.listdir(path)]

../../cropped_images/canada.png
../../cropped_images/korea_south.png
../../cropped_images/united_states_of_america.png
../../cropped_images/japan.png


In [60]:
points = [
    ["720 Briar Hill Ave", 43.706843167968444, -79.43343763536379, "https://i.imgur.com/qHeXMDn.jpg", "Your travelling spirit seems to have rubbed off on me", "ca"],
    ["The Big Apple", 44.06580278508552, -77.91405655898555, "https://i.imgur.com/ly25nux.jpg", "Quite a LOVEly roadside attraction", "ca"],
    ["Tea Ceremony", 35.711900, 139.792800, "https://i.imgur.com/W7Po077.jpg", "You can really pull off a kimono!", "japan"],
    ["Tokyo Skytree", 35.709400, 139.808600, "https://i.imgur.com/g5xFE4w.jpg", "I love you more than corn soup", "japan"],
    ["Box Burger", 35.256200, 139.048000, "https://i.imgur.com/jlhhitN.jpg", "This photo is really sweet", "japan"],
    ["Kiyomizu-dera", 34.994600, 135.785400, "https://i.imgur.com/Npa3uYv.jpg", "", "japan"],
    ["Masuyacho", 34.997400, 135.781100, "https://i.imgur.com/RO81Kc8.jpg", "", "japan"],
    ["Hiroshima Dreamination", 34.388200, 132.461200, "https://i.imgur.com/i4RVUMD.jpg", "", "japan"],
    ["Miyajima", 34.279200, 132.325100, "https://i.imgur.com/QnYCUoG.jpg", "", "japan"],
    ["Hahoe Folk Village", 36.538600, 128.517000, "https://i.imgur.com/3tDDosH.jpg", "", "korea_south"],
    ["Third Infiltration Tunnel, DMZ", 37.91671648025547, 126.69829308650752, "https://i.imgur.com/t4y01pG.jpg", "", "korea_south"],
    ["Bukchon Hanok Village", 37.5828025947213, 126.98361926829193, "https://i.imgur.com/X9bGRho.jpg", "", "korea_south"],
    ["The Ex", 43.635400, -79.413600, "https://i.imgur.com/OinvUP5.jpg", "", "canada"],
    ["Eataly", 43.66977878442531, -79.38867336311971, "https://i.imgur.com/NOLqlIJ.jpg", "", "canada"],
    ["SkyDome", 43.640947, -79.389594, "https://i.imgur.com/LIf3OMh.jpg", "", "canada"],
    ["Science Centre", 43.716245366555604, -79.34016622715741, "https://i.imgur.com/ngmTlTA.jpeg", "", "canada"],
]

maybes = [
    ["Belmanor", 44.526464496121, -80.35017367446154, "https://i.imgur.com/OlHUw0d.jpg", "", "canada"],
    ["The Melting Point", 34.862678, -111.785881, "https://i.imgur.com/YaMTx4v.jpg", "", "united_states_of_america"],
    ["The Guggenheim", 40.782936, -73.959124, "https://i.imgur.com/MbSRl11.jpg", "", "united_states_of_america"],
    ["Alpine", 44.51520990123986, -80.35446874749427, "https://i.imgur.com/EY1OCO4.jpg", "", "canada"],
    ["Odawara Station", 35.255500, 139.156000, "https://i.imgur.com/yEGPE79.jpg", "", "japan"],
]

markers = [
    ["720 Briar Hill Ave", 43.706843167968444, -79.43343763536379, "Your travelling spirit seems to have rubbed off on me", "ca"],
    ["The Big Apple", 44.06580278508552, -77.91405655898555, "Quite a LOVEly roadside attraction", "ca"],
    ["Tea Ceremony", 35.711900, 139.792800, "You can really pull off a kimono!", "jp"],
    ["Tokyo Skytree", 35.709400, 139.808600, "I love you more than corn soup", "jp"],
    ["Box Burger", 35.256200, 139.048000, "This photo is really sweet", "jp"],
    ["Kiyomizu-dera", 34.994600, 135.785400, "", "jp"],
    ["Masuyacho", 34.997400, 135.781100, "", "jp"],
    ["Hiroshima Dreamination", 34.388200, 132.461200,"", "jp"],
    ["Miyajima", 34.279200, 132.325100, "", "jp"],
    ["Hahoe Folk Village", 36.538600, 128.517000, "", "kr"],
    ["Third Infiltration Tunnel, DMZ", 37.91671648025547, 126.69829308650752, "", "kr"],
    ["Bukchon Hanok Village", 37.5828025947213, 126.98361926829193, "", "kr"],
    ["The Ex", 43.635400, -79.413600, "", "ca"],
    ["Eataly", 43.66977878442531, -79.38867336311971, "", "ca"],
    ["SkyDome", 43.640947, -79.389594, "", "ca"],
    ["Science Centre", 43.716245366555604, -79.34016622715741, "", "ca"],
    ["Belmanor", 44.526464496121, -80.35017367446154, "", "ca"],
    ["The Melting Point", 34.862678, -111.785881, "", "us"],
    ["The Guggenheim", 40.782936, -73.959124, "", "us"],
    ["Alpine", 44.51520990123986, -80.35446874749427, "", "ca"],
    ["Odawara Station", 35.255500, 139.156000, "", "jp"],
]

In [81]:
df = pd.DataFrame(points + maybes, columns = ['location', 'latitude', 'longitude', "image", "caption", "country"])
df['hover_info'] = df.apply(lambda row: 
                            f'''<h4 style = "text-align: center; font-family: Futura; font-weight: bold;">{row["location"]}</h4>'''\
                            f'''<img src = "https://github.com/Logan-Daniels/Happy-Mothers-Day/blob/main/images/{row['location'].replace(" ", "_").lower()}.jpg?raw=true"'''\
                            f'''alt = "{row['location']}"style="max-width: 300px; max-height: 300px;">'''\
                            f'''<p style = "text-align: center; font-family: streetwear; font-size: 14px;">{row["caption"]}</p>''', 
                             axis =1)
df = df.drop(columns = ['location', 'image', 'caption'])

In [89]:
for filename in os.listdir('../../images'):
    filename = '../../images/' + filename
    os.rename(filename, filename.replace(' ', '_').lower())

In [82]:
df[df['country'] == 'canada'].style

,latitude,longitude,country,hover_info
0,44.065803,-77.914057,canada,The Big AppleQuite a LOVEly roadside attraction
11,43.635400,-79.413600,canada,The Ex
12,43.669779,-79.388673,canada,Eataly
13,43.640947,-79.389594,canada,SkyDome
14,43.716245,-79.340166,canada,Science Centre
15,43.706843,-79.433438,canada,720_Briar_Hill_AveYour travelling spirit seems to have rubbed off on me
16,44.526464,-80.350174,canada,Belmanor
19,44.515210,-80.354469,canada,Alpine


In [52]:
TORONTO_COORDINATES = (43.66977878442531, -79.38867336311971)
m = folium.Map(location = TORONTO_COORDINATES, 
               zoom_start = 12,
               min_zoom = 2,
               max_bounds=True)

tile_provider = xyz.MapBox
tile_provider['accessToken'] = MAPBOX_API_KEY
tile_provider['url'] = 'https://api.mapbox.com/styles/v1/logandaniels/cluryho1d01wt01pbdqtxdp21/tiles/{z}/{x}/{y}{r}?access_token=' + MAPBOX_API_KEY
tile_provider['attribution'] = ' '
tile_provider['html_attribution'] = ' '
tile_provider['min_zoom'] = '2'

folium.TileLayer(
    tiles = tile_provider.build_url(api_key = MAPBOX_API_KEY),
    attr = tile_provider['attribution'],
    name = tile_provider['name'],
    max_zoom = tile_provider['max_zoom'],
    detect_retina = True,
    min_zoom = tile_provider['min_zoom']
).add_to(m)

headers = {'User-Agent': 'Mozilla/5.0'}

for marker in range(len(df)):
    url = "https://raw.githubusercontent.com/Logan-Daniels/Happy-Mothers-Day/main/cropped_images/{}.png".format
    url = url(df['country'].iloc[marker])
    icon = folium.CustomIcon(
        url,
        icon_size = (62, 104),
        icon_anchor = (31, 104),
        popup_anchor = (0, -100),
    )

    folium.Marker(
        location = [df['latitude'].iloc[marker], df['longitude'].iloc[marker]], 
        icon = icon, 
        popup = df['hover_info'].iloc[marker]
    ).add_to(m)
m

In [31]:
m.save('mothers_day.html')

In [76]:
TORONTO_COORDINATES = (43.66977878442531, -79.38867336311971)
m = folium.Map(location = TORONTO_COORDINATES, 
               zoom_start = 12,
               min_zoom = 2,
               max_bounds=True)

tile_provider = xyz.MapBox
tile_provider['accessToken'] = MAPBOX_API_KEY
tile_provider['url'] = 'https://api.mapbox.com/styles/v1/logandaniels/cluryho1d01wt01pbdqtxdp21/tiles/{z}/{x}/{y}{r}?access_token=' + MAPBOX_API_KEY
tile_provider['attribution'] = ' '
tile_provider['html_attribution'] = ' '
tile_provider['min_zoom'] = '2'

folium.TileLayer(
    tiles = tile_provider.build_url(api_key = MAPBOX_API_KEY),
    attr = tile_provider['attribution'],
    name = tile_provider['name'],
    max_zoom = tile_provider['max_zoom'],
    detect_retina = True,
    min_zoom = tile_provider['min_zoom']
).add_to(m)

headers = {'User-Agent': 'Mozilla/5.0'}

for marker in range(len(df)):
    url = "https://raw.githubusercontent.com/Logan-Daniels/Happy-Mothers-Day/main/cropped_images/{}.png".format
    url = url(df['country'].iloc[marker])
    icon = folium.CustomIcon(
        url,
        icon_size = (62, 104),
        icon_anchor = (31, 104),
        popup_anchor = (0, -100),
    )

    folium.Marker(
        location = [df['latitude'].iloc[marker], df['longitude'].iloc[marker]], 
        icon = icon, 
        popup = df['hover_info'].iloc[marker]
    ).add_to(m)
m